In [27]:
import pandas as pd
news = pd.read_csv(r'D:\Downloads\combined_data (1).csv')

In [28]:
news.head()

,ID,dates,name,age,"duration, months",сondition,overall rating,reviews
0,1,2023-05-10,Ginkgo,3-6,NaN,NaN,2.7,NaN
1,2,2023-05-10,Ginkgo,3-6,NaN,NaN,2.7,NaN
2,3,2023-05-10,Ginkgo,3-6,NaN,NaN,2.7,NaN
3,4,2023-01-31,Ginkgo,65-74,<1,Tinnitus,2.7,No effect on tinnitus
4,5,2022-03-11,Ginkgo,55-64,1-6,Peripheral arterial disease (PAD),4.3,"Take gingo for the pain in my legs, it works. ..."


In [29]:
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

news = news.drop('overall rating', axis=1)
news['reviews'] = news['reviews'].fillna('')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim


def get_lda_objects(text):
    nltk.download('stopwords')    
    stop=set(stopwords.words('english'))

    
    def _preprocess_text(text):
        corpus=[]
        stem=PorterStemmer()
        lem=WordNetLemmatizer()
        for news in text:
            words=[w for w in word_tokenize(news) if (w not in stop)]

            words=[lem.lemmatize(w) for w in words if len(w)>2]

            corpus.append(words)
        return corpus
    
    corpus=_preprocess_text(text)
    
    dic=gensim.corpora.Dictionary(corpus)
    bow_corpus = [dic.doc2bow(doc) for doc in corpus]
    
    lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 4, 
                                   id2word = dic,                                    
                                   passes = 10,
                                   workers = 2)
    
    return lda_model, bow_corpus, dic

def plot_lda_vis(lda_model, bow_corpus, dic):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dic)
    return vis

In [31]:
lda_model, bow_corpus, dic = get_lda_objects(news['reviews'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lyuob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
lda_model.show_topics()

[(0,
  '0.018*"taking" + 0.014*"take" + 0.012*"pain" + 0.012*"level" + 0.011*"week" + 0.009*"day" + 0.007*"Vit" + 0.006*"effect" + 0.006*"Vitamin" + 0.006*"started"'),
 (1,
  '0.010*"week" + 0.009*"day" + 0.009*"take" + 0.008*"taking" + 0.007*"month" + 0.006*"oil" + 0.006*"n\'t" + 0.005*"year" + 0.005*"also" + 0.005*"started"'),
 (2,
  '0.023*"hair" + 0.013*"taking" + 0.011*"day" + 0.008*"n\'t" + 0.007*"month" + 0.007*"nail" + 0.007*"year" + 0.007*"Biotin" + 0.006*"started" + 0.006*"take"'),
 (3,
  '0.014*"day" + 0.008*"take" + 0.007*"year" + 0.007*"n\'t" + 0.007*"taking" + 0.006*"sleep" + 0.006*"effect" + 0.006*"pain" + 0.006*"week" + 0.005*"silver"')]

In [33]:
plot_lda_vis(lda_model, bow_corpus, dic)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.071794  0.039689       1        1  29.531308
0     -0.083162 -0.004637       2        1  25.747442
3      0.030836 -0.077875       3        1  22.500525
1     -0.019468  0.042823       4        1  22.220725, topic_info=         Term        Freq       Total Category  logprob  loglift
723      hair  258.000000  258.000000  Default  30.0000  30.0000
646     level  153.000000  153.000000  Default  29.0000  29.0000
5434   Biotin   74.000000   74.000000  Default  28.0000  28.0000
649       oil  103.000000  103.000000  Default  27.0000  27.0000
2138     nail   90.000000   90.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
179   vitamin   23.384392  112.433013   Topic4  -5.8169  -0.0661
175      time   25.893853  165.278012   Topic4  -5.7149  -0.3495
3445   silver   21.973401   99.844243   Topic4  -5.8791  -0.0096
490    result   20.560698   73.475329   Topic4  -5.9456   0.2306
17        use   21.958669  113.065085   Topic4  -5.8798  -0.1347

[336 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
129       1  0.613596    've
129       2  0.115049    've
129       3  0.076700    've
129       4  0.204532    've
574       1  0.267679    ...
...     ...       ...    ...
376       4  0.136212  would
124       1  0.328338   year
124       2  0.239598   year
124       3  0.257346   year
124       4  0.173043   year

[610 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])